#### TFT API 에 대해서 설명되어있는 사이트 : https://velog.io/@twa02189/TFT-API-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%84%A4%EB%AA%85

In [4]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

#### 기본 키 저장 + baser URL 저장 + request header 저장

In [5]:
# api_key 저장
api_key = 'RGAPI-39d1115b-559c-49b0-bacb-cd3ea0588e94' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

#### 챌린저, 마스터, 그랜드 마스터 구간 유저들의 닉네임 추출 

In [6]:
# challenger

challenger_url = base_url + 'league/v1/challenger'

challenger = requests.get(challenger_url, headers = request_header).json()

# grand master

grandmaster_url = base_url + 'league/v1/grandmaster'

grandmaster = requests.get(grandmaster_url, headers = request_header).json()

# master

master_url = base_url + 'league/v1/master'

master = requests.get(master_url, headers = request_header).json()

In [7]:
# 가져온 마스터 데이터 프레임 조회 (마스터 랭크에 속한 유저 조회)
pd.DataFrame(challenger).head(3)

# 가져온 그랜드마스터 데이터 프레임 조회 (그랜드마스터 랭크에 속한 유저 조회)
pd.DataFrame(grandmaster).head(3)

# 가져온 마스터 데이터 프레임 조회 (마스터 랭크에 속한 유저 조회)
pd.DataFrame(master).head(3)

,tier,leagueId,queue,name,entries
0,GRANDMASTER,f1e46b7f-5703-3dad-b9c5-6a8f5dfa5598,RANKED_TFT,Soraka's Enforcers,"{'summonerId': 'VVpwkqbuN3VVwG378diAEHea6jFrq1ADYBQAEstZizthf7k', 'summonerName': '이렇게 좋은 날', 'leaguePoints': 329, 'rank': 'I', 'wins': 93, 'losses': 50, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"
1,GRANDMASTER,f1e46b7f-5703-3dad-b9c5-6a8f5dfa5598,RANKED_TFT,Soraka's Enforcers,"{'summonerId': 'KKtx2l6oWWPWxU6Pi5tm0trZQI6yFkVbHjyMALGori8cG24DSiXPdy5gug', 'summonerName': '천연만화', 'leaguePoints': 418, 'rank': 'I', 'wins': 74, 'losses': 32, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"
2,GRANDMASTER,f1e46b7f-5703-3dad-b9c5-6a8f5dfa5598,RANKED_TFT,Soraka's Enforcers,"{'summonerId': 'UF2IeZC87PAxxS0F7xg6VQNADum-YqoBLfYCOAb0LdmQL7WIfnII-3Cm1Q', 'summonerName': 'kscv05', 'leaguePoints': 248, 'rank': 'I', 'wins': 88, 'losses': 51, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"


In [8]:
# master_df 에 저장
challenger_df = pd.DataFrame(challenger)

# master_df 에 저장
master_df = pd.DataFrame(master)

# grandmaster_df 에 저장
grandmaster_df = pd.DataFrame(grandmaster)

##### 결측치 제거 및 json normalize 테스트

In [9]:
# entries 컬럼 확장 
master_entries_normalized = json_normalize(master_df['entries'])

# 확장된 데이터프레임 병합
master_df = pd.concat([master_df.drop(['entries'], axis=1), master_entries_normalized], axis=1)

# 필요없는 컬럼 drop('queue', 'name')
master_df = master_df.drop(['leagueId', 'queue', 'name', 'rank'], axis=1) ## 이 네가지는 당연하고 필요없는 정보

# summonerName 컬럼이 결측치인 행 제거
master_df = master_df.dropna(subset=['summonerName'])

# 확인
master_df ## null 값이 아니라 공란 그자체가 값으로 존재하는 듯함

# summonerName 컬럼이 공란인 행 제거
master_df = master_df[master_df['summonerName'] != '']

# 확인
master_df.head() # 결측치 제거된 것 확인했음

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,MASTER,98aXU2A44FuIOOV27snPKqkHXGm6VocL-1m9rPMb6_Rjkw,덕소나몽이,0,96,85,False,False,True,False
1,MASTER,RtwaM2fHJQGrhrEJO6JMtopi4jEGZWZ4M4JB_mNYtes_rlg,iliillliillii1il,101,71,36,False,False,True,False
2,MASTER,VAKsMSSE117CotJorkQgMzC1taWY3hm0mc0azpz28nAzeqg,1Chon,107,71,37,False,False,True,True
3,MASTER,yu3ysnF3p8f8yFitlLc1vrg8akquavQKXWWtFgzkT3syIRI,guiId,138,64,44,False,False,True,False
4,MASTER,9YoBNg8hHk_u_acXGwSHedDsKUCt-hvVDNTXZRAhM9g2Jew,리 셉,57,63,51,False,False,True,True


##### 결측치 제거 및 json 펼치는 함수 생성

In [10]:
def json_normalize_and_dropna(df):
    # entries 컬럼 확장
    entries_normalized = json_normalize(df['entries'])
    
    # 확장된 데이터프레임 병합
    df = pd.concat([df.drop(['entries'], axis=1), entries_normalized], axis=1)
    
    # 필요없는 컬럼 drop('queue', 'name')
    df = df.drop(['leagueId', 'queue', 'name', 'rank'], axis=1)
    
    # summonerName 컬럼이 결측치인 행 제거
    df = df.dropna(subset=['summonerName'])
    
    # summonerName 컬럼이 공란인 행 제거
    df = df[df['summonerName'] != '']
    
    return df

In [11]:
# grandmaster df 결측치 제거 및 json normalize
grandmaster_df = json_normalize_and_dropna(grandmaster_df)

#### 추출한 데이터 프레임에서 닉네임을 모두 뽑아서 리스트로 작성

##### 닉네임 하나 골라서 puuid 추출 테스트

In [12]:
## 한명의 닉네임을 넣어서 제대로 나오는지 확인

# 닉네임을 넣어줄 url 저장
id_url = base_url + "summoner/v1/summoners/by-name/COPY O"

# 닉네임을 넣었을 때, 반환값 저장
random_id = requests.get(id_url, headers = request_header).json()

# puuid 추출
random_id['puuid'] ## puuid 추출 확인 / random_id 는 딕셔너리 형태(json)

't_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw'

##### 앞서서 만든 master_df 에서 닉네임만 리스트로 추출 (df 에서 닉네임만 리스트로 추출하는 함수 생성)

In [13]:
# 닉네임 리스트 추출 함수(닉네임 공란도 제거)
def get_summoner_names_list(df):
    df = df[df['summonerName'] != '']
    summoner_names_list = df['summonerName'].tolist()
    return summoner_names_list

# master_df 의 닉네임 리스트 추출
master_summoner_names_list = get_summoner_names_list(master_df)

# grandmaster_df 의 닉네임 리스트 추출
grandmaster_summoner_names_list = get_summoner_names_list(grandmaster_df)

# 확인
print(master_summoner_names_list[:3])

print(grandmaster_summoner_names_list[:3])

['덕소나몽이', 'iliillliillii1il', '1Chon']
['이렇게 좋은 날', '천연만화', 'kscv05']


##### 닉네임의 리스트를 넣으면 puuid 의 리스트로 반환하는 함수 생성

In [14]:
# 마스터 puuid 리스트 추출 함수
def get_puuid_list(nickname_list):
    puuid_list = []
    
    for nickname in nickname_list:
        # 닉네임을 넣어줄 url 저장
        id_url = base_url + f"summoner/v1/summoners/by-name/{nickname}"
        
        # 닉네임을 넣었을 때, 반환값 저장
        random_id = requests.get(id_url, headers=request_header).json()
        
        # puuid 추출하여 리스트에 추가
        if 'puuid' in random_id:
           puuid_list.append(random_id['puuid'])
    
    return puuid_list

# 마스터 puuid 리스트 추출해서 master_puuid_list 에 저장
master_puuid_list = get_puuid_list(master_summoner_names_list)

# 그랜드 puuid 리스트 추출
grandmaster_puuid_list = get_puuid_list(grandmaster_summoner_names_list)

In [15]:
# 확인
print(master_puuid_list[:3])

print(grandmaster_puuid_list[:3])

['kYhP68i62rIsuHRjgj08nyudp6z25E2T-ccqE-zYl3i_qkfznXO_7YhmFqZNp6M4bL7fTW1nYb83JQ', '8upXuKqJvCQvEmuMiyeHDtXJ7Qkj3Gl4hwFhqorDSVVUs2AL7hcLoqtdTgraRGxtJ8QSnBZQx1qubw', 'QaBjEGmoD-QWTZ7sF4Q6C2VkwIc_481e2feDOU8QHOC3ndbF2ceWuwzMnyvPvAaLu3NrZx_hA_gT7w']
['rx1nmyxEddDyQujL2nvjfiHV_LDcBEo-zyf16l3BFnXQExOMoB6tNss-WGlc_GRgcac2tg1GhDgBhg', 'lskGGafPGWafTOrBtnSVRCyYdggWRybYvU_B0r65qQvnGCU2C4gmZmt9rMDAnruQuoloPyWnpeBiqQ', 'mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ']


#### 추출한 puuid 리스트를 이용하여 GAME ID 추출

##### 시험용 닉네임과 puuid 를 이용하여 게임 데이터 조회 후 한개의 게임 데이터를 조회(테스트)

In [16]:
# 앞에서 생성했던 시험용 닉네임과 puuid 를 이용해서 game id 추출 테스트

random_puu_id = random_id['puuid']

random_game_url = "https://Asia.api.riotgames.com/tft/" + f'match/v1/matches/by-puuid/{random_puu_id}/ids?count=20'

random_game_id = requests.get(random_game_url, headers = request_header).json()

# 확인
random_game_id

['KR_7004133580',
 'KR_7003836228',
 'KR_7003780848',
 'KR_7003725466',
 'KR_7003636575',
 'KR_7003576286',
 'KR_7003496678',
 'KR_7003415677',
 'KR_7003339946',
 'KR_7003289547',
 'KR_7003237553',
 'KR_7003190560',
 'KR_7003121826',
 'KR_7003089848',
 'KR_7003060002',
 'KR_7003019623',
 'KR_7003002191',
 'KR_7002969109',
 'KR_7002933292',
 'KR_7002843922']

In [28]:
# 리스트에서 첫번째 GAME ID 를 선택
random_first_game_id = random_game_id[0]

# 첫번쨰 GAEM ID 를 API 를 통해 게임 데이터 조회
random_game_result_url = "https://Asia.api.riotgames.com/tft/" + f"match/v1/matches/{random_first_game_id}"

# 조회한 결과를 game_result 에 저장
random_game_result = requests.get(random_game_result_url, headers=request_header).json()

# 확인
print(type(random_game_result)) #json타입 확인

print(random_game_result)

<class 'dict'>
{'metadata': {'data_version': '5', 'match_id': 'KR_7004133580', 'participants': ['7fwBtBreUky8V3NXRDc8R3ora47tJpR9E6Dy5puhlRiwX5v6xtTtkZmpVQWYTQ32fqjX6o9brdnAnw', 'f9B0WwvLVCQioMzQY50uhZb6ecPXvvkexY29FMX2QrvFgC_ulkRM08Iwvy0tt2Z_ecSHkxcMAtmn7A', 't_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw', 'LZ6lv1UG0xUMVNyzu-R4xTNkHietPz1rLIdevEkkzaTzQc15A3FZvJR1BnDIVnDvXJHcRay84sZCQw', 'tJFb_m_KhxcX1PO8CTZemLkqx3k5N-c7ZxDdI_yvnPDmQ-xnclrXE0dr_SVLmr6SDDKWXLTL35jl1w', 'GEvycnX2BCHPJvYWxovKW0UaUjBhJ1lzWQVjmlMG5_8oV83TMr8Id-HYx-TXt7maxBDPvdu-E9p9JQ', 'LeGm9atIqxWUGhgqVJHQeZ3Sl38LfOIGLIfRtD_RZ2x-PFfPWVYwOMB235aQFpXZZQmCcmgMR-_fEw', 'i1YaFAt-zVs_YI4ag1WM8xjV4SYx0ThPYNAmr8GHYcj8TQOsQDHvCgLbxpH1TRgy5i6sZXkQ6T-Fpg']}, 'info': {'endOfGameResult': 'GameComplete', 'gameCreation': 1711513967000, 'gameId': 7004133580, 'game_datetime': 1711516380537, 'game_length': 2401.256103515625, 'game_version': 'Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14

In [19]:
# 딕셔너리 안에서 info key 에 속하는 딕셔너리를 dataframe 으로 만들기
game_result_df = pd.DataFrame(random_game_result['info'])

In [20]:
participants = pd.DataFrame(random_game_result['info']['participants'])

participants[:1]  ## 한게임에 8명이 참가하므로 총 8개의 데이터가 생기는 것을 확인

,augments,companion,gold_left,last_round,level,missions,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
0,"[TFT9_Augment_YouHaveMySword, TFT9_Augment_JeweledLotus, TFT9_Augment_CyberneticBulk3]","{'content_ID': '695aeccf-59ba-40d3-be5c-0459548e7891', 'item_ID': 52012, 'skin_ID': 12, 'species': 'PetBunny'}",0,28,8,"{'Assists': 0, 'DamageDealt': 0, 'DamageDealtToObjectives': 50614, 'DamageDealtToTurrets': 3, 'DamageTaken': 0, 'Deaths': 0, 'DoubleKills': 0, 'GoldEarned': 0, 'GoldSpent': 0, 'InhibitorsDestroyed': 0, 'KillingSprees': 8, 'Kills': 0, 'LargestKillingSpree': 22896, 'LargestMultiKill': 118106, 'MagicDamageDealt': 0, 'MagicDamageDealtToChampions': 0, 'MagicDamageTaken': 0, 'NeutralMinionsKilledTeamJungle': 0, 'PentaKills': 1, 'PhysicalDamageDealt': 0, 'PhysicalDamageDealtToChampions': 19, 'PhysicalDamageTaken': 1, 'PlayerScore0': 6, 'PlayerScore1': 7, 'PlayerScore10': 0, 'PlayerScore11': 0, 'PlayerScore2': 83, 'PlayerScore3': 0, 'PlayerScore4': 1, 'PlayerScore5': 1, 'PlayerScore6': 9, 'PlayerScore9': 0, 'QuadraKills': 0, 'Spell1Casts': 0, 'Spell2Casts': 0, 'Spell3Casts': 0, 'Spell4Casts': 1668, 'SummonerSpell1Casts': 0, 'TimeCCOthers': 0, 'TotalDamageDealtToChampions': 0, 'TotalMinionsKilled': 0, 'TripleKills': 0, 'TrueDamageDealt': 1, 'TrueDamageDealtToChampions': 66, 'TrueDamageTaken': 7, 'UnrealKills': 0, 'VisionScore': 0, 'WardsKilled': 67}",6,0,7fwBtBreUky8V3NXRDc8R3ora47tJpR9E6Dy5puhlRiwX5v6xtTtkZmpVQWYTQ32fqjX6o9brdnAnw,1662.605713,69,"[{'name': 'TFT11_Altruist', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Bruiser', 'num_units': 4, 'style': 2, 'tier_current': 2, 'tier_total': 4}, {'name': 'TFT11_Dragonlord', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_Fortune', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Ghostly', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_InkShadow', 'num_units': 2, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Lovers', 'num_units': 1, 'style': 5, 'tier_current': 1, 'tier_total': 1}, {'name': 'TFT11_Storyweaver', 'num_units': 3, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Trickshot', 'num_units': 4, 'style': 3, 'tier_current': 2, 'tier_total': 2}, {'name': 'TFT11_Umbral', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}]","[{'character_id': 'TFT11_Sivir', 'itemNames': [], 'name': '', 'rarity': 0, 'tier': 2}, {'character_id': 'TFT11_Aatrox', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Riven', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Teemo', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Galio', 'itemNames': ['TFT_Item_WarmogsArmor', 'TFT_Item_GargoyleStoneplate'], 'name': '', 'rarity': 4, 'tier': 1}, {'character_id': 'TFT11_Kaisa', 'itemNames': ['TFT_Item_GuinsoosRageblade', 'TFT_Item_InfinityEdge'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Sylas', 'itemNames': ['TFT_Item_Bloodthirster', 'TFT_Item_TitansResolve', 'TFT_Item_TitansResolve'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Xayah', 'itemNames': [], 'name': '', 'rarity': 6, 'tier': 1}, {'character_id': 'TFT11_Kayle', 'itemNames': ['TFT11_Storyweaver3_Support'], 'name': '', 'rarity': 9, 'tier': 1}]"


In [23]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   augments                 8 non-null      object 
 1   companion                8 non-null      object 
 2   gold_left                8 non-null      int64  
 3   last_round               8 non-null      int64  
 4   level                    8 non-null      int64  
 5   missions                 8 non-null      object 
 6   placement                8 non-null      int64  
 7   players_eliminated       8 non-null      int64  
 8   puuid                    8 non-null      object 
 9   time_eliminated          8 non-null      float64
 10  total_damage_to_players  8 non-null      int64  
 11  traits                   8 non-null      object 
 12  units                    8 non-null      object 
dtypes: float64(1), int64(6), object(6)
memory usage: 960.0+ bytes


In [21]:
# 게임 참가자 중 첫번째 유저가 활성화한 시너지의 정보 조회

random_first_user_trait = pd.DataFrame(participants.loc[0, 'traits'])

random_first_user_trait # num_units 는 유닛수, tier_current / tier_total 총 몇단계까지 있는 시너지에서 현재 몇단계까지 켰는지

,name,num_units,style,tier_current,tier_total
0,TFT11_Altruist,1,0,0,3
1,TFT11_Bruiser,4,2,2,4
2,TFT11_Dragonlord,1,0,0,4
3,TFT11_Fortune,1,0,0,3
4,TFT11_Ghostly,1,0,0,4
5,TFT11_InkShadow,2,0,0,3
6,TFT11_Lovers,1,5,1,1
7,TFT11_Storyweaver,3,1,1,4
8,TFT11_Trickshot,4,3,2,2
9,TFT11_Umbral,1,0,0,4


In [22]:
# 첫번쨰 유저가 게임 종료시 가지고 있던 유닛 조회

random_first_user_unit = pd.DataFrame(participants.loc[0, 'units'])

random_first_user_unit # 캐릭터에게 준 아이템과 몇성인지 조회 가능 / rarity 는 희귀도로 코스트와는 다른값

,character_id,itemNames,name,rarity,tier
0,TFT11_Sivir,[],,0,2
1,TFT11_Aatrox,[],,1,2
2,TFT11_Riven,[],,1,2
3,TFT11_Teemo,[],,1,2
4,TFT11_Galio,"[TFT_Item_WarmogsArmor, TFT_Item_GargoyleStoneplate]",,4,1
5,TFT11_Kaisa,"[TFT_Item_GuinsoosRageblade, TFT_Item_InfinityEdge]",,4,2
6,TFT11_Sylas,"[TFT_Item_Bloodthirster, TFT_Item_TitansResolve, TFT_Item_TitansResolve]",,4,2
7,TFT11_Xayah,[],,6,1
8,TFT11_Kayle,[TFT11_Storyweaver3_Support],,9,1
